In [1]:
from aocd import get_data

# Day 9: Disk Fragmenter

## Part One

In [2]:
import numpy as np

example_input = "2333133121414131402"
counts = np.array(list(example_input), dtype=int); counts

array([2, 3, 3, 3, 1, 3, 3, 1, 2, 1, 4, 1, 4, 1, 3, 1, 4, 0, 2])

In [3]:
len(counts)

19

First we need to convert the input into the file representation:

In [4]:
file_values = np.arange(0, len(list(example_input))//2 + 1)
empty_values = np.full(len(list(example_input))//2 + 1, '.')

In [5]:
import itertools
interleaved = np.array(list(itertools.chain(*zip(file_values, empty_values))), dtype=str); interleaved

array(['0', '.', '1', '.', '2', '.', '3', '.', '4', '.', '5', '.', '6',
       '.', '7', '.', '8', '.', '9', '.'], dtype='<U21')

In [6]:
blocks = np.repeat(interleaved[:len(counts)], counts); blocks

array(['0', '0', '.', '.', '.', '1', '1', '1', '.', '.', '.', '2', '.',
       '.', '.', '3', '3', '3', '.', '4', '4', '.', '5', '5', '5', '5',
       '.', '6', '6', '6', '6', '.', '7', '7', '7', '.', '8', '8', '8',
       '8', '9', '9'], dtype='<U21')

In [7]:
''.join(blocks)

'00...111...2...333.44.5555.6666.777.888899'

#### Apprach 1:

Now we have our file representation we need to actually do the work of defragging

In [8]:
def shift_blocks(arr):
    while '.' in arr:
        value = arr[-1]  # get last element 
        arr = np.delete(arr, -1)  # remove last element, numpy doesn't have pop()
        
        dot_index = np.where(arr == '.')[0][0]
        arr[dot_index] = value
    
    return arr
        

In [9]:
blocks = shift_blocks(blocks); blocks

array(['0', '0', '9', '9', '8', '1', '1', '1', '8', '8', '8', '2', '7',
       '7', '7', '3', '3', '3', '6', '4', '4', '6', '5', '5', '5', '5',
       '6', '6'], dtype='<U21')

In [10]:
positions = np.arange(0, len(blocks))
checksum = np.sum(positions * blocks.astype(int)); checksum

np.int64(1928)

#### Approach 2:

`shift_blocks` isn't efficient for a larger dataset - we can do better

In [11]:
def replace_dots_with_reversed(arr):
    # reverse array and remove dots
    reversed_array = arr[::-1][arr[::-1] != '.']
    # find all indices for dots and replace
    dot_indices = np.where(arr == '.')
    arr[dot_indices] = reversed_array[:len(arr[dot_indices])]
    # num of ints in array should be the same as originally
    return arr[:len(reversed_array)]

In [12]:
test_array = np.array(['1','.','.','2','.','3','4'])

In [13]:
replace_dots_with_reversed(test_array)

array(['1', '4', '3', '2'], dtype='<U1')

In [14]:
replace_dots_with_reversed(blocks)

array(['0', '0', '9', '9', '8', '1', '1', '1', '8', '8', '8', '2', '7',
       '7', '7', '3', '3', '3', '6', '4', '4', '6', '5', '5', '5', '5',
       '6', '6'], dtype='<U21')

In [15]:
data = get_data(day=9, year=2024)

In [16]:
import itertools

def calculate(data):
    # set up data
    counts = np.array(list(data), dtype=int)
    file_values = np.arange(0, len(data)//2 + 1)
    empty_values = np.full(len(data)//2 + 1, '.')
    interleaved = list(itertools.chain(*zip(file_values, empty_values)))
    blocks = np.repeat(interleaved[:len(counts)], counts)

    # run "defragger"
    blocks = replace_dots_with_reversed(blocks)
    
    positions = np.arange(0, len(blocks))
    checksum = np.sum(positions * blocks.astype(int))
    
    return checksum

In [17]:
calculate(data)

np.int64(6370402949053)

## Part Two

The eager amphipod already has a new plan: rather than move individual blocks, he'd like to try compacting the files on his disk by moving whole files instead.

This time, attempt to move whole files to the leftmost span of free space blocks that could fit the file. Attempt to move each file exactly once in order of decreasing file ID number starting with the file with the highest file ID number. If there is no span of free space to the left of a file that is large enough to fit the file, the file does not move.

Start over, now compacting the amphipod's hard drive using this new method instead. What is the resulting filesystem checksum?



In [18]:
class Block:
    b_id: int
    length: int
    tail: int

    def __init__(self, b_id, length, tail):
        self.b_id = int(b_id)
        self.length = int(length)
        self.tail = int(tail)

    def __repr__(self):
        return f"{self.b_id}" * self.length + "." * self.tail

    def remove_tailspace(self):
        "When a file is placed immediately after this one, the tailspace is removed"
        self.tail = 0

    def reduce_tailspace(self, length):
        self.tail -= length

    def check_sum(self, offset):
        end_offset = (offset + self.length) - 1
        factor = int(
            (
                (offset + end_offset)
                * ((end_offset - offset) + 1)
            ) / 2
        )
        return int(self.b_id) * factor
        

Now use the above to represent our example input:

In [19]:
counts

array([2, 3, 3, 3, 1, 3, 3, 1, 2, 1, 4, 1, 4, 1, 3, 1, 4, 0, 2])

In [20]:
interleaved

array(['0', '.', '1', '.', '2', '.', '3', '.', '4', '.', '5', '.', '6',
       '.', '7', '.', '8', '.', '9', '.'], dtype='<U21')

In [21]:
zipped = list(zip(counts, interleaved))

files = []
# loop through two elements at a time
for i in range(0, len(zipped), 2):
    file_length, file_val = zipped[i]
    space_length, _ = zipped[i + 1] if i + 1 < len(zipped) else (0, None)
    files.append(Block(file_val, file_length, space_length))

In [22]:
def defrag_blocks(blocks):
    
    i = len(blocks) - 1
    
    while i > 0:
        source = blocks[i]
        to_move = source.length
        
        for j in range(i):
            dest = blocks[j]
            
            if dest.tail >= to_move:
                moved = Block(source.b_id, to_move, dest.tail - to_move)
                blocks.insert(j + 1, moved)
                i += 1

                dest.remove_tailspace()

                del blocks[i]
                
                blocks[i - 1].tail += source.length + source.tail

                break
        i -= 1

    return blocks

In [23]:
defrag_blocks(files)

[00, 99, 2, 111, 777., 44., 333...., 5555., 6666....., 8888..]

'00992111777.44.333....5555.6666.....8888..'

In [24]:
''.join([str(f) for f in files])

'00992111777.44.333....5555.6666.....8888..'

In [25]:
def checksum(blocks):
    c_sum = 0
    offset = 0
    for block in blocks:
        c_sum += int(block.check_sum(offset))
        offset += int(block.length) + int(block.tail)
    return c_sum

In [26]:
files

[00, 99, 2, 111, 777., 44., 333...., 5555., 6666....., 8888..]

In [27]:
checksum(files)

2858

In [29]:
counts = np.array(list(data), dtype=int)
file_values = np.arange(0, len(data)//2 + 1)
empty_values = np.full(len(data)//2 + 1, '.')
interleaved = list(itertools.chain(*zip(file_values, empty_values)))

In [32]:
zipped = list(zip(counts, interleaved))

files = []
# loop through two elements at a time
for i in range(0, len(zipped), 2):
    file_length, file_val = zipped[i]
    space_length, _ = zipped[i + 1] if i + 1 < len(zipped) else (0, None)
    files.append(Block(file_val, file_length, space_length))

In [ ]:
defrag_blocks(files)

In [35]:
checksum(files)

6398096697992